# Обработка данных сборников Росстата

### Цель:
Найти оптимальный способ выгрузки и организации данных из статитических сборников Росстата, для их удобной обработки и анализа. 

### Задачи:
1. Создать перечень из 5-7 показателей в разрезе субъектов Российский Федерации, для каждоной из тематик: "здравоохранение" и "демография". Указать источники. (xls)
2. Сформировать датасеты в формате временных рядов по выбранным показателям (csv)
3. Провести предобработку данных 
4. Создать id для увязки датасетов
5. Обозначить проблемы объединения данных

## Установка всех необходимых пакетов и библиотек

In [ ]:
pip install -U click
pip install xlwt
pip install python-docx
pip install docx2csv

In [1]:
import pandas as pd
from docx import Document
from docx.shared import Inches
import click
import xlwt
from docx2csv import extract_tables, extract
import unicodedata
import requests
from io import BytesIO
import re

## Выгрузка данных из Росстата

Были взяты данные из статистического сборника "Регионы России. Социально-экономические показатели - 2019 г." расположенного по адресу: https://rosstat.gov.ru/bgd/regl/b19_14p/Main.htm

В качестве основных демографических показателей были выбраны следующие:
* Численность населения
* Удельный вес городского и сельского населения в общей численности населения
* Коэффициенты младенческой смертности
* Соотношение браков и разводов
* Смертность населения в трудоспособном возрасте
* Ожидаемая продолжительность жизни при рождении

Из категории здравоохранение были выбраны такие категрии:
* Число больничных коек
* Численность врачей всех специальностей
* Прерывание беременности (аборты)
* Заболеваемость на 1000 человек населения
* Мощность амбулаторно-поликлинических организаций
* Численность среднего медицинского персонала

Выгрузка демографических данных

In [167]:
population_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-01.docx").content))
urban_rural_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-03.docx").content))
child_mort_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-12.docx").content))
marriages_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-18.docx").content))
work_age_mort_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-10.docx").content))
lifespan_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-15.docx").content))

Выгрузка данных категории здравоохранение

In [314]:
hosp_beds_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/06-01.docx").content))
doctors_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/06-04.docx").content))
abortion_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/06-07.docx").content))
clinics_capacity_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/06-03.docx").content))
nursing_staff_docx = extract_tables(BytesIO(requests.get("https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/06-06.docx").content))

## Сборка pd.DataFrame из .docx для Демографии

Далее, для раздела демография я создаю три траблицы. Количество таблиц определилось их спецификой. Например, данные по урбанизации и продолжительности жизни содержат большьшее число данных, поэтому пришлось их вынести в отдельные таблицы. Данные вытягивались из docx файла, и через индексацию перераспределены и собраны в удобные для анализа таблицы pandas.

### Таблица: Демография
(Численность населения, детская смертность, брачность, смертность в трудоспособном возрасте)

In [256]:
demography_df = pd.DataFrame(columns=['date','region','population', 'child_mort', 'marriages', 'work_age_mort'])

table = population_docx

for i in range(2): 
    for j in range(10):
        for k in range(len(table[i])-1):
            
            if bool(unicodedata.normalize("NFKD", table[i][k+1][j+1].decode('utf-8')).strip()):
                date = table[i][0][j+1].decode('utf-8')
                region = table[i][k+1][0].decode('utf-8')
                population = table[i][k+1][j+1].decode('utf-8')
                child_mort = child_mort_docx[i][k+1][j+1].decode('utf-8')
                marriages = marriages_docx[i][k+1][j+1].decode('utf-8')
                
                work_age_mort = work_age_mort_docx[i][k+1][j+1].decode('utf-8')                
                work_age_mort = re.sub(',', '.', work_age_mort)
                
                demography_df = demography_df.append({
                            'date': date, 
                            'region': region, 
                            'population': population,
                            'child_mort': child_mort,
                            'marriages': marriages,
                            'work_age_mort': work_age_mort}, 
                            ignore_index=True)

### Таблица: Урбанизация

In [223]:
urban_rural_df = pd.DataFrame(columns=[])

table = urban_rural_docx

for i in range(2): 
    for j in range(14):
        for k in range(len(table[i])-2):
            
            if bool(unicodedata.normalize("NFKD", table[i][k+2][j+1].decode('utf-8')).strip()):
                date = table[i][1][j+1].decode('utf-8')
                region = table[i][k+2][0].decode('utf-8')
                
                if table[i][k+2][j+1].decode('utf-8') != '–':
                    urban_rural_val = table[i][k+2][j+1].decode('utf-8')
                    urban_rural_val = re.sub(',', '.', urban_rural_val)
                else:
                    urban_rural_val = 0
                
                urban_rural_stat = table[i][0][j+1].decode('utf-8')

                urban_rural_df = urban_rural_df.append({
                            'date': date, 
                            'region': region, 
                            'urban_rural_stat': urban_rural_stat,
                            'urban_rural_val': urban_rural_val}, 
                            ignore_index=True)

### Таблица: Продолжительность жизни

In [274]:
lifespan_df = pd.DataFrame(columns=[])

table = lifespan_docx

for i in range(6): 
    for j in range(10):
        for k in range(len(table[i])-2):
            
            if bool(unicodedata.normalize("NFKD", table[i][k+2][j+1].decode('utf-8')).strip()):
                date = table[i][0][j+1].decode('utf-8')
                region = table[i][k+2][0].decode('utf-8')
                if any(c.isalpha() for c in table[i][k+2][j+1].decode('utf-8'))*1 == 0:
                    if table[i][k+2][j+1].decode('utf-8') != '–':
                        lifespan_val = table[i][k+2][j+1].decode('utf-8')
                        lifespan_val = re.sub(',', '.', lifespan_val)
                    else:
                        lifespan_val = 0
                    if i%2 == 0:
                        lifespan_sex = table[i][1][2].decode('utf-8')

                    lifespan_df = lifespan_df.append({
                                'date': date, 
                                'region': region, 
                                'lifespan_val': lifespan_val,
                                'lifespan_sex': lifespan_sex},
                                ignore_index=True)   

Проверяю данные 

In [261]:
demography_df.head()

,date,region,population,child_mort,marriages,work_age_mort
0,2005-01-01,Российская Федерация,143236,143236.0,567,827.8
1,2005-01-01,Центральный федеральный округ,38109,38109.0,576,793.7
2,2005-01-01,Белгородская область,1512,1512.0,503,618.6
3,2005-01-01,Брянская область,1327,1327.0,575,1021.3
4,2005-01-01,Владимирская область,1486,1486.0,570,1040.3


In [262]:
urban_rural_df.head()

,date,region,urban_rural_stat,urban_rural_val
0,2005-01-01,Российская Федерация,Городское население,73.2
1,2005-01-01,Центральный федеральный округ,Городское население,80.4
2,2005-01-01,Белгородская область,Городское население,65.6
3,2005-01-01,Брянская область,Городское население,68.3
4,2005-01-01,Владимирская область,Городское население,77.2


In [263]:
lifespan_df.head()

,date,lifespan_sex,lifespan_val,region
0,2005-01-01,Все население,65.37,Российская Федерация
1,2005-01-01,Все население,66.50,Центральный федеральный округ
2,2005-01-01,Все население,68.43,Белгородская область
3,2005-01-01,Все население,63.40,Брянская область
4,2005-01-01,Все население,63.51,Владимирская область


## Сборка pd.DataFrame из .docx для Здравоохранения

### Таблица: Здравоохранение

In [351]:
data_health = pd.DataFrame(columns=['date','region','stat','hosp_beds','doctors','abortion', 'clinics_capacity', 'nursing_staff'])

table = hosp_beds_docx

for i in range(2): 
    for j in range(14):
        for k in range(len(table[i])-2):
            
            if bool(unicodedata.normalize("NFKD", table[i][k+2][j+1].decode('utf-8')).strip()):
                date = table[i][1][j+1].decode('utf-8')
                region = table[i][k+2][0].decode('utf-8')
                if any(c.isalpha() for c in table[i][k+2][j+1].decode('utf-8'))*1 == 0:
                    if table[i][k+2][j+1].decode('utf-8') != '…':
                        hosp_beds = table[i][k+2][j+1].decode('utf-8')
                        hosp_beds = re.sub(',', '.', hosp_beds)
                        
                        doctors = doctors_docx[i][k+2][j+1].decode('utf-8')
                        doctors = re.sub(',', '.', doctors)
                        
                        abortion = abortion_docx[i][k+2][j+1].decode('utf-8')
                        abortion = re.sub(',', '.', abortion)
                        
                        clinics_capacity = clinics_capacity_docx[i][k+2][j+1].decode('utf-8')
                        clinics_capacity = re.sub(',', '.', clinics_capacity)
                        
                        nursing_staff = nursing_staff_docx[i][k+2][j+1].decode('utf-8')
                        nursing_staff = re.sub(',', '.', nursing_staff)
                        
                    else:
                        hosp_beds = 0
                        doctors = 0
                        abortion = 0
                        clinics_capacity = 0
                        nursing_staff = 0
                        
                    stat = re.sub(' коек', '', hosp_beds_docx[i][0][j+1].decode('utf-8'))
                    
                    data_health = data_health.append({
                                'date': date, 
                                'region': region, 
                                'stat': stat,
                                'hosp_beds': hosp_beds,
                                'doctors': doctors,
                                'abortion': abortion,
                                'clinics_capacity': clinics_capacity,
                                'nursing_staff': nursing_staff}, 
                                ignore_index=True)

Проверка данных

In [352]:
data_health.head()

,date,region,stat,hosp_beds,doctors,abortion,clinics_capacity,nursing_staff
0,2005,Российская Федерация,"Всего, тыс.",1575.4,690.3,43,3637.9,1529.8
1,2005,Центральный федеральный округ,"Всего, тыс.",428.2,193.9,34,1024.9,389.4
2,2005,Белгородская область,"Всего, тыс.",16.9,6.0,31,33.9,17.8
3,2005,Брянская область,"Всего, тыс.",16.9,4.8,47,29.8,15.1
4,2005,Владимирская область,"Всего, тыс.",15.5,5.0,46,44.6,15.1


## Передобработка данных

Устанавливаю соответсвующие типы данных

In [275]:
demography_df['date'] = pd.to_datetime(demography_df['date'], format='%Y')
demography_df['population'] = demography_df['population'].astype('int')
demography_df['child_mort'] = demography_df['population'].astype('float')
demography_df['marriages'] = demography_df['marriages'].astype('int')
demography_df['work_age_mort'] = demography_df['work_age_mort'].astype('float')

lifespan_df['date'] = pd.to_datetime(lifespan_df['date'], format='%Y')
lifespan_df['lifespan_val'] = lifespan_df['lifespan_val'].astype('float')

urban_rural_df['date'] = pd.to_datetime(urban_rural_df['date'], format='%Y')
urban_rural_df['urban_rural_val'] = urban_rural_df['urban_rural_val'].astype('float')

In [353]:
data_health['date'] = pd.to_datetime(data_health['date'], format='%Y')
data_health['hosp_beds'] = data_health['hosp_beds'].astype('float')
data_health['doctors'] = data_health['doctors'].astype('float')
data_health['abortion'] = data_health['abortion'].astype('int')
data_health['clinics_capacity'] = data_health['clinics_capacity'].astype('float')
data_health['nursing_staff'] = data_health['nursing_staff'].astype('float')

Проверяю данные

In [258]:
demography_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           950 non-null    datetime64[ns]
 1   region         950 non-null    object        
 2   population     950 non-null    int32         
 3   child_mort     950 non-null    float64       
 4   marriages      950 non-null    int32         
 5   work_age_mort  950 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int32(2), object(1)
memory usage: 37.2+ KB


In [259]:
lifespan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2816 entries, 0 to 2815
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          2816 non-null   datetime64[ns]
 1   lifespan_sex  2816 non-null   object        
 2   lifespan_val  2816 non-null   float64       
 3   region        2816 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 88.1+ KB


In [260]:
urban_rural_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              1336 non-null   datetime64[ns]
 1   region            1336 non-null   object        
 2   urban_rural_stat  1336 non-null   object        
 3   urban_rural_val   1336 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 41.9+ KB


In [354]:
data_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              1336 non-null   datetime64[ns]
 1   region            1336 non-null   object        
 2   stat              1336 non-null   object        
 3   hosp_beds         1336 non-null   float64       
 4   doctors           1336 non-null   float64       
 5   abortion          1336 non-null   int32         
 6   clinics_capacity  1336 non-null   float64       
 7   nursing_staff     1336 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int32(1), object(2)
memory usage: 78.4+ KB


## Проблема объединения данных

Проверим количество срок в каждой из таблиц

In [359]:
print(len(demography_df))
print(len(lifespan_df))
print(len(urban_rural_df))
print(len(data_health))

950
2816
1336
1336


Как видно длина у таблиц разная, что указывает на то, что не получится просто собрать их все в одну. Можно попробовать объединить их по бщему ключу, например дате и региону, но тогда могут возникнуть проблемы удвения данных. Несмотря на то что число строк для последних вдух таблиц одинаково, их все равно нельзя объединить без удвоения данных, поскольку данные по урбенизации разбиты по категориям город/село, а данные по здоровью разбиты по все тыс / на 10 тыс челоек.

Объедининение данных таблицы по плотности городского / сельского населения с основной таблицой с демографическими данными.

In [273]:
data1 = demography_df
data2 = lifespan_df
data3 = urban_rural_df
data4 = data_health

data = data1.merge(data3,on=['date', 'region'])
data.head()

,date,region,population,child_mort,marriages,work_age_mort,urban_rural_stat,urban_rural_val
0,2005-01-01,Российская Федерация,143236,143236.0,567,827.8,Городское население,73.2
1,2005-01-01,Российская Федерация,143236,143236.0,567,827.8,Сельское население,26.8
2,2005-01-01,Центральный федеральный округ,38109,38109.0,576,793.7,Городское население,80.4
3,2005-01-01,Центральный федеральный округ,38109,38109.0,576,793.7,Сельское население,19.6
4,2005-01-01,Белгородская область,1512,1512.0,503,618.6,Городское население,65.6


Как видно данные удваиваются. Точно так же будет, если объединять таблицу с продолжительньностью жизни для разных полов и общей таблицей. Лучше оставить данные в трех отдельных таблицах. Не вижу необходимости дополнительно создавать индексы, когда есть названия регионов и годы - их можно использовать для объединения таблиц, по необходимости.

## Заключение

Мною были собраны данные из статистического сборника "Регионы России. Социально-экономические показатели - 2019 г." расположенного по адресу: https://rosstat.gov.ru/bgd/regl/b19_14p/Main.htm 
Я выбрал шесть показателей из раздела Демография и пять показателей из раздела Здоровье. Данные в сборнике представлены в неудобном для анализа виде - в docx файле в формате таблицы. Для того, чтобы достать оттуда данные, я использовал ряд библиотек из отрытых источников, в т.ч. на гитхаб. Полученные данные были неструктурированы и представляли собой набор символов, которые необходимо было отформатировать, почитисть и привести в читаемый вид. Кроме того, мною была продумана система организации данных таким образом, чтобы их было удобно агрегировать. Код полностью воспроизводим. Полученные алгоритмы выгрузки могут быть использованы для дальнейшей работы и частичной автоматизации процеса работы с данными из государственных источников данных. 

### Вспомогательный код
Необходим для теста новых данных

In [ ]:
table = doctors_docx

for i in range(2): 
    for j in range(14):
        for k in range(len(table[i])-2):
            
            if bool(unicodedata.normalize("NFKD", table[i][k+2][j+1].decode('utf-8')).strip()):
                if any(c.isalpha() for c in table[i][k+2][j+1].decode('utf-8'))*1 == 0:
                    print(table[i][k+2][j+1].decode('utf-8'))

 

In [ ]:
hosp_beds[0][1][1].decode('utf-8')

In [ ]:
data = pd.DataFrame(columns=[])

table = hosp_beds_docx

for i in range(2): 
    for j in range(10):
        for k in range(len(table[i])-1):
            
            if bool(unicodedata.normalize("NFKD", table[i][k+1][j+1].decode('utf-8')).strip()):
                date = table[i][0][j+1].decode('utf-8')
                region = table[i][k+1][0].decode('utf-8')
                value = table[i][k+1][j+1].decode('utf-8')

                data = data.append({
                            'date': date, 
                            'region': region, 
                            'population': value}, 
                            ignore_index=True)
                
data